In [1]:
import cv2
import numpy as np
import os
import easyocr

# Configuration variables
UPLOAD_FOLDER = "C:/workspace_project/AIproject/workspace/recieved_images"
MODEL_STORAGE_DIRECTORY = "C:/workspace_project/AIproject/workspace/user_network_dir"
BLOCKED_CHARACTERS = "<>|]\:[+=`@#$%^&*;'"

# Ensure upload and model directories exist
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(MODEL_STORAGE_DIRECTORY, exist_ok=True)

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    # blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    # denoised_image = cv2.medianBlur(blurred_image, 5)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # equalized_image = cv2.equalizeHist(gray_image)
    # adaptive_threshold_image = cv2.adaptiveThreshold(
    #     equalized_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    # )
    return gray_image


def extract_text_from_image(image_path, filename, languages=["ko"]):
    model_storage_directory = os.path.abspath(MODEL_STORAGE_DIRECTORY)
    user_network_directory = os.path.abspath(MODEL_STORAGE_DIRECTORY)

    # Custom model 사용
    reader_custom = easyocr.Reader(
        lang_list=languages,
        gpu=True,
        model_storage_directory=model_storage_directory,
        user_network_directory=user_network_directory,
        recog_network="custom"
    )
    # 기본 model 사용
    reader_default = easyocr.Reader(lang_list=languages, gpu=True)

    preprocessed_image = preprocess_image(image_path)
    result_custom = reader_custom.readtext(preprocessed_image, blocklist=BLOCKED_CHARACTERS)
    result_default = reader_default.readtext(preprocessed_image, blocklist=BLOCKED_CHARACTERS)

    # bbox 단위로 결과 비교 및 선택
    result = []
    for custom, default in zip(result_custom, result_default):
        if custom[2] > default[2]:  # custom model의 confidence가 더 높은 경우
            result.append(custom)
        else:
            result.append(default)

    extracted_data = []
    loaded_image = cv2.imread(image_path)
    for bbox, text, confidence in result:
        extracted_data.append({"text": text, "confidence": confidence})
        top_left = tuple(map(int, bbox[0]))
        bottom_right = tuple(map(int, bbox[2]))
        cv2.rectangle(loaded_image, top_left, bottom_right, (255, 0, 0), 2)
        text_position = (top_left[0], top_left[1] - 10)
        cv2.putText(
            loaded_image,
            f"{confidence*100:.2f}%",
            text_position,
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (255, 0, 0),
            2,
        )

    result_image_filename = os.path.join(UPLOAD_FOLDER, filename)
    cv2.imwrite(result_image_filename, loaded_image)

    return extracted_data, result_image_filename

# 예제 실행
def main():
    test_image_path = "C:/workspace_project/AIproject/workspace/demo_images/04.jpg"
    filename = os.path.basename(test_image_path)

    extracted_data, result_image_path = extract_text_from_image(
        test_image_path, filename
    )
    if extracted_data:
        print("Extracted Texts:")
        for data in extracted_data:
            print(f"{data['text']} ({data['confidence']*100:.2f}%)")
        print(f"Annotated image saved at {result_image_path}")
    else:
        print("No text found in image.")

if __name__ == "__main__":
    main()

Extracted Texts:
cns (71.24%)
귀하 (98.84%)
0038 (78.40%)
책 (28.74%)
뻐웨웨뻐 (4.46%)
디디 (88.22%)
떼 (39.47%)
정평화 지하 가 1호 (11.22%)
젠 (63.50%)
H.P? 01-48g9-B070 (8.60%)
010-B8B-2050 (31.44%)
빼 (86.12%)
ID? dodream2o5o (21.36%)
빼a 빠 (13.36%)
빼 빠 (31.21%)
신상마켓 I0i 디디OB) (9.92%)
2-0-만(원) (4.50%)
500 (99.58%)
000 (89.90%)
품명 (25.32%)
단가 (79.16%)
수량 (96.97%)
금액 (85.52%)
빼베 (12.06%)
2C000 (70.61%)
120,0CO (14.17%)
비비포켓배기그러이 (9.39%)
20,0 (41.46%)
빼 (78.27%)
비비포켓빼기그러이 (11.06%)
20,000 (48.22%)
빼 (55.15%)
비비포켓하기베이지g8 (9.33%)
20,000 (56.59%)
80,000 (49.86%)
뻐 (61.92%)
60,00 (54.73%)
벤뱃뀌 (3.66%)
20,000 (99.20%)
60,o0 (29.10%)
판매소계 (95.21%)
6권 (49.43%)
25 (99.01%)
빼 (89.29%)
햄 (40.83%)
뽑 (11.36%)
500 (98.73%)
0oo (44.39%)
금입 (35.03%)
500 (90.61%)
000 (84.61%)
당 (99.21%)
잔 (95.91%)
0 (99.95%)
뻗션O (5.64%)
뻐h음부째랜 (3.29%)
"세금계산서E 매합 5입 마라g니다. (4.26%)
(뷰 썹 (13.63%)
"색, 아o보_계업 갈라교만 안매모 (2.63%)
250765909666555 (57.64%)
Annotated image saved at C:/workspace_project/AIproject/workspace/recieved_images\04.jpg
